# NBA Champion Prediction 2021 #

It's another season of NBA, teams are soon to be locked in for playoffs and getting eliminated but nothing has been set in stone yet. With the data of each NBA teams from each season, including stats such as free throw percentage, field goal percentage, three point percentage, I decided it would be fun to use past NBA season to predict the NBA champion of season 2021.


In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from sklearn.preprocessing import StandardScaler


The NBA stats since 1980 - May 1st, 2021, retrieved from [Kaggle](https://www.kaggle.com/sumitrodatta/nba-aba-baa-stats), cleaned and combined with past championship winning record in R using tidyverse.

In [2]:
past_data = pd.read_csv('nba_champs_comb1.csv')
past_data.head()

data_2021 = pd.read_csv('sea21.csv')
data_2021.head()

,season,lg.x,team,abbreviation.x,playoffs.x,g,mp,fg,fga,fg_percent,...,orb_percent,ft_fga,opp_e_fg_percent,opp_tov_percent,opp_drb_percent,opp_ft_fga,arena,attend,attend_g,wpct
0,2021,NBA,Atlanta Hawks,ATL,False,64,15485,2575,5579,0.462,...,24.6,0.227,0.531,11.2,77.4,0.194,State Farm Arena,37810,1304,0.531250
1,2021,NBA,Boston Celtics,BOS,False,64,15460,2648,5677,0.466,...,24.9,0.186,0.538,12.9,77.9,0.212,TD Garden,22980,696,0.531250
2,2021,NBA,Chicago Bulls,CHI,False,63,15220,2664,5570,0.478,...,22.0,0.160,0.539,11.5,80.3,0.199,United Center,0,0,0.412698
3,2021,NBA,Charlotte Hornets,CHO,False,62,14930,2480,5418,0.458,...,23.1,0.182,0.551,13.4,75.8,0.162,Spectrum Center,40239,1388,0.483871
4,2021,NBA,Cleveland Cavaliers,CLE,False,63,15245,2452,5408,0.453,...,23.7,0.189,0.553,13.6,76.4,0.186,Quicken Loans Arena,66588,2220,0.333333


Dropping the column not needed for the code, these variables are random effects and are not being considered in the computation 

In [3]:
final_team = data_2021['team']
dropping = ['season', 'lg.x', 'team', 'abbreviation.x', 'playoffs.x', 'g', 'lg.y', 'playoffs.y', 'arena', 'attend', 'attend_g', 'w', 'l', 'pw', 'pl', 'abbreviation.y', 'mp', 'fg', 'fga', 'x3p', 'x3pa', 'x2p', 'x2pa', 'ft', 'fta', 'orb', 'drb', 'trb', 'stl', 'pace','f_tr','x3p_ar','ts_percent','e_fg_percent','orb_percent','ft_fga','opp_e_fg_percent','opp_tov_percent','opp_drb_percent','opp_ft_fga']
past_data = past_data.drop(dropping, axis=1)
data_2021 = data_2021.drop(dropping, axis=1)

past_data_y = past_data.pop('champs')
data_2021.pop('champs')

print(past_data.shape)
print(data_2021.shape)

(1134, 18)
(30, 18)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(past_data, past_data_y, test_size=0.33, random_state=0)

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

Input shape: [18]


In [5]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(759, 18)
(759,)
(375, 18)
(375,)


In [7]:
model = keras.Sequential([
    layers.Dense(6, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(6, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(6, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1),
])

In [8]:
model.compile(
    optimizer='sgd',
    loss='mae',
    metrics=['accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    epochs=1000,
)

Epoch 1/1000
24/24 [==============================] - 1s 8ms/step - loss: 0.8453 - accuracy: 0.7101 - val_loss: 0.2761 - val_accuracy: 0.8000
Epoch 2/1000
24/24 [==============================] - 0s 1ms/step - loss: 0.5868 - accuracy: 0.7773 - val_loss: 0.2048 - val_accuracy: 0.8827
Epoch 3/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.4228 - accuracy: 0.8590 - val_loss: 0.1609 - val_accuracy: 0.9227
Epoch 4/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.2885 - accuracy: 0.8933 - val_loss: 0.1190 - val_accuracy: 0.9467
Epoch 5/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.2216 - accuracy: 0.9183 - val_loss: 0.0921 - val_accuracy: 0.9547
Epoch 6/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.1707 - accuracy: 0.9368 - val_loss: 0.0837 - val_accuracy: 0.9573
Epoch 7/1000
24/24 [==============================] - 0s 2ms/step - loss: 0.1282 - accuracy: 0.9460 - val_loss: 0.0742 - val_accuracy: 0.9573
Epoch 

In [9]:
pred_2021 = model.predict(data_2021)

In [10]:
final_2021 = pd.DataFrame(final_team)
final_2021['champ'] = pred_2021
final_2021 = final_2021.sort_values(by=['champ'])
final_2021.to_csv('final_ML.csv', index = False)